# Film list scraping

In [ ]:
# URL of the webpage to scrape
film_url = "https://www.yorck.de/en/films"

# Send a GET request to the webpage
film_response = requests.get(film_url)
film_soup = BeautifulSoup(film_response.content, "html.parser")

film_titles = film_soup.select("h3.MuiTypography-h3.whatson-film-title")

clean_titles = []

# Extract and store the clean film titles
for title in film_titles:
    clean_title = title.get_text(strip=True)
    clean_titles.append(clean_title)

# print(clean_titles) 

# IMDB API key

In [ ]:
secrets_file = open("imdb_api_key.txt")
api_key = secrets_file.read()
api_key

# Film table building using the IMDB API
There is a more usesful function lower down called "Collecting film data" which will work for target and current films.
Will leave this one for longevity and because it still works fine, just requires addiitonal steps

In [ ]:
# import requests
# import time
# import pandas as pd

# # RapidAPI details
# api_host = "imdb8.p.rapidapi.com"

# # Base URL for search
# search_url = "https://imdb8.p.rapidapi.com/v2/search"
# # Base URL for getting ratings
# ratings_url = "https://imdb8.p.rapidapi.com/title/v2/get-ratings"

# # Headers for the API requests
# headers = {
#     "X-RapidAPI-Key": api_key,
#     "X-RapidAPI-Host": api_host
# }

# # List to store the results
# results = []

# # Loop through the list of titles
# for title in clean_titles:
#     # Query string for the search API
#     querystring = {"searchTerm": title, "type": "MOVIE", "first": "1", "country": "US", "language": "en-US"}

#     # Make the API call to search for the title
#     response = requests.get(search_url, headers=headers, params=querystring)
    
#     # Adding delay between API calls
#     time.sleep(1)
    
#     if response.ok:
#         try:
#             data = response.json()
#             edges = data["data"]["mainSearch"]["edges"]
#             film_id = None
#             image_url = None
#             for edge in edges:
#                 node = edge.get("node", {})
#                 entity = node.get("entity", {})
#                 if entity.get("__typename") == "Title":
#                     film_id = entity.get("id", "")
#                     primary_image = entity.get("primaryImage", {})
#                     image_url = primary_image.get("url", "")
#                     break
#         except (KeyError, json.JSONDecodeError) as e:
#             print(f"Error processing search response for {title}: {e}")
#             film_id = None
#             image_url = None
#     else:
#         print(f"Error: {response.status_code} - {response.reason}")
#         film_id = None
#         image_url = None
    
#     if film_id is not None:
#         querystring2 = {"tconst": film_id, "country": "US", "language": "en-US"}

#         response2 = requests.get(ratings_url, headers=headers, params=querystring2)
        
#         # Adding delay between API calls
#         time.sleep(5)
        
#         if response2.ok:
#             try:
#                 data2 = response2.json()
#                 aggregate_rating = data2["data"]["title"]["ratingsSummary"]["aggregateRating"]
#             except (KeyError, json.JSONDecodeError) as e:
#                 print(f"Error processing ratings response for {film_id}: {e}")
#                 aggregate_rating = None
#         else:
#             print(f"Error: {response2.status_code} - {response2.reason}")
#             aggregate_rating = None
        
#         # Store the result in the results list
#         results.append({
#             "Title": title,
#             "Film ID": film_id,
#             "Image URL": image_url,
#             "Aggregate Rating": aggregate_rating
#         })
#     else:
#         print(f"Film ID not found for {title}")

# # Create a DataFrame from the results
# rated_films = pd.DataFrame(results)

# # filter the responses
# top_rated_films = rated_films[rated_films['Aggregate Rating'] > 6.5]

# top_rated_films

 # Creating the museum dataframe

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pandas as pd

history_museum = 'https://www.museumsportal-berlin.de/en/program?exhibition_type=special&category=geschichte-kulturgeschichte'
art_museum = 'https://www.museumsportal-berlin.de/en/program?exhibition_type=special&category=architektur-kunstgewerbe-design&category=architektur-kunstgewerbe-design&category=architektur-kunstgewerbe-design&category=architektur-kunstgewerbe-design&category=architektur-kunstgewerbe-design&category=bildende-kunst'
design_museum = 'https://www.museumsportal-berlin.de/en/program?exhibition_type=special&category=architektur-kunstgewerbe-design'

def scrape_museum_data(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    
    driver.get(url)

    # Scroll and load more data
    for _ in range(10):  # Adjust the range for the number of scrolls
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(5)  # Adjust the sleep time to allow data to load

    # Get the page source after scrolling
    html = driver.page_source
    driver.quit()

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Initialize lists to store the extracted data
    types = []
    locations = []
    h2_titles = []
    h3_titles = []
    times = []

    # Loop through each card and extract the necessary information
    for card in soup.find_all('mp-card'):
        # Extract the type
        card_type = card.find('p', class_='mp-card-type').get_text(strip=True)
        types.append(card_type)
        
        # Extract the location
        location = card.find('p', class_='mp-card-location').get_text(strip=True)
        locations.append(location)
        
        # Extract the h2 title
        h2_title = card.find('h2').get_text(strip=True)
        h2_titles.append(h2_title)
        
        # Extract the h3 title
        h3_title = card.find('h3')
        h3_titles.append(h3_title.get_text(strip=True) if h3_title else "")
        
        # Extract the time
        time_element = card.find('time')
        times.append(time_element.get_text(strip=True) if time_element else "Closed this day")

    # Create a DataFrame from the extracted data
    df = pd.DataFrame({
        'Type': types,
        'Museum': locations,
        'Name': h2_titles,
        'Description': h3_titles,
        'time': times
    })

    return df

# List of URLs to scrape
urls = [
    history_museum,  # Replace with actual URL
    art_museum,  # Replace with actual URL
    design_museum   # Replace with actual URL
]

# Scrape each URL and store the resulting DataFrames in a list
dataframes = [scrape_museum_data(url) for url in [history_museum, art_museum, design_museum]]

hist_exhib = dataframes[0]
art_exhib = dataframes[1]
design_exhib = dataframes[2]

value1 = 'design'
design_exhib['tag'] = value1
# design_exhib
value2 = 'art'
art_exhib['tag'] = value2
# art_exhib
value3 = 'history'
hist_exhib['tag'] = value3
# hist_exhib

combined_museums = pd.concat([design_exhib, art_exhib, hist_exhib], ignore_index=True)
# combined_museums

duplicate_rows = combined_museums[combined_museums.duplicated(subset=['h2'])]
# duplicate_rows
tagged_museum_recs = duplicate_rows.drop_duplicates(subset=['h2'])
clean_museum_recs = tagged_museum_recs[tagged_museum_recs['time'] != 'Closed this day']
# clean_museum_recs

# This really has to be updated to include only the months you need. There must be better code for this and to include only one month rather than excluding all
search_strings = ['July', 'August', 'September','October','November','December', ['children', 'kinder']]

# Create a regex pattern to match any of the search strings
pattern = '|'.join(search_strings)

# Filter rows where 'description' contains any of the search strings
open_museum_recs = clean_museum_recs[~clean_museum_recs['time'].str.contains(pattern, case=False, na=False)]

# open_museum_recs

# Save museum recs to json

In [ ]:
open_museum_recs.to_json('museum_recs.json', orient='records')

# Create events df

In [ ]:
events_url = "https://www.berlin.de/events/"

# Send a GET request to the webpage
events_response = requests.get(events_url)
events_soup = BeautifulSoup(events_response.content, "html.parser")

# Initialize lists to store the extracted data
titles = []
dates = []
descriptions = []
links = []
images = []

# Loop through each article and extract the necessary information
for article in events_soup.find_all('article', class_='modul-teaser'):
    # Extract the title
    title_tag = article.find('h3', class_='title')
    title = title_tag.get_text(strip=True) if title_tag else 'No title'
    titles.append(title)
    
    # Extract the date
    date_tag = article.find('p', class_='teaser__meta')
    date = date_tag.get_text(strip=True) if date_tag else 'No date'
    dates.append(date)
    
    # Extract the description
    description_tag = article.find('p', class_='text')
    description = description_tag.get_text(strip=True).replace(' mehr', '') if description_tag else 'No description'
    descriptions.append(description)
    
    # Extract the link
    link_tag = title_tag.find('a') if title_tag else None
    link = link_tag['href'] if link_tag else 'No link'
    links.append(link)
    
    # Extract the image URL
    image_tag = article.find('div', class_='image__image').find('img')
    image = image_tag['src'] if image_tag else 'No image'
    images.append(image)

# Create a DataFrame from the extracted data
events_df = pd.DataFrame({
    'Title': titles,
    'Date': dates,
    'Description': descriptions,
    'Link': links,
    'Image': images
})

# Display the DataFrame
# events_df

current_events_df = events_df[events_df['Date'] != 'No date']

# Creating the target events 

In [ ]:
target_events_url = "https://www.berlin.de/events/top-10/"

# Send a GET request to the webpage
tg_events_response = requests.get(target_events_url)
target_events_soup = BeautifulSoup(tg_events_response.content, "html.parser")

# Initialize lists to store the extracted data
titles = []
dates = []
descriptions = []
links = []
images = []

# Loop through each article and extract the necessary information
for article in target_events_soup.find_all('article', class_='modul-teaser'):
    # Extract the title
    title_tag = article.find('h3', class_='title')
    title = title_tag.get_text(strip=True) if title_tag else 'No title'
    titles.append(title)
    
    # Extract the date
    date_tag = article.find('p', class_='teaser__meta')
    date = date_tag.get_text(strip=True) if date_tag else 'No date'
    dates.append(date)
    
    # Extract the description
    description_tag = article.find('p', class_='text')
    description = description_tag.get_text(strip=True).replace(' mehr', '') if description_tag else 'No description'
    descriptions.append(description)
    
    # Extract the link
    link_tag = title_tag.find('a') if title_tag else None
    link = link_tag['href'] if link_tag else 'No link'
    links.append(link)
    
    # Extract the image URL
    image_tag = article.find('div', class_='image__image').find('img')
    image = image_tag['src'] if image_tag else 'No image'
    images.append(image)

# Create a DataFrame from the extracted data
target_events_df = pd.DataFrame({
    'Title': titles,
    'Date': dates,
    'Description': descriptions,
    'Link': links,
    'Image': images
})

target_events_df = target_events_df[target_events_df['Date'] != 'No date']
# target_events_df

# Building similarity score for current events vs the target

In [ ]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

nlp = spacy.load('de_core_news_sm')

# Function to preprocess text: tokenize, lemmatize, and remove stop words
def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)

# Apply preprocessing to the descriptions
current_events_df['processed_description'] = current_events_df['Description'].apply(preprocess)
target_events_df['processed_description'] = target_events_df['Description'].apply(preprocess)

# Combine descriptions with target descriptions
all_descriptions = current_events_df['processed_description'].tolist() + target_events_df['processed_description'].tolist()

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform all descriptions
tfidf_matrix = vectorizer.fit_transform(all_descriptions)

# Separate the tfidf matrix into event descriptions and target descriptions
event_tfidf_matrix = tfidf_matrix[:len(current_events_df)]
target_tfidf_matrix = tfidf_matrix[len(current_events_df):]

# Compute cosine similarity between event descriptions and target descriptions
cosine_sim = cosine_similarity(event_tfidf_matrix, target_tfidf_matrix)

# Function to get max similarity score for each event description
def get_max_similarity_score(cosine_sim_row):
    return max(cosine_sim_row)

# Iterate through each row and append max similarity score as a new column
current_events_df['Max_Similarity_Score'] = [get_max_similarity_score(row) for row in cosine_sim]

# Drop the processed_description column as it's not needed for the final output
current_events_df = current_events_df.drop(columns=['processed_description'])

# Display the DataFrame with the new column
# current_events_df

threshold = 0.1
recommended_events = current_events_df[current_events_df['Max_Similarity_Score'] > threshold]

# Display the filtered DataFrame
# recommended_events

# Translate events table and add urls for buttons

In [ ]:
from googletrans import Translator

# Initialize the Translator
translator = Translator()

# Translate the contents of the 'text' column to Spanish
recommended_events['translated_text'] = recommended_events['Description'].apply(lambda x: translator.translate(x, src='de', dest='en').text)
recommended_events['translated_date'] = recommended_events['Date'].apply(lambda x: translator.translate(x, src='de', dest='en').text)
recommended_events['translated_title'] = recommended_events['Title'].apply(lambda x: translator.translate(x, src='de', dest='en').text)


base_url = "https://www.berlin.de"

# Append the base URL to each row in the 'link' and 'image' columns
recommended_events['Link'] = base_url + recommended_events['Link']
recommended_events['Image'] = base_url + recommended_events['Image']

# Creating target film data list

In [ ]:
base_url = "https://letterboxd.com/lukeinha27/films/rated/3.5-5/page/"

# Initialize an empty list to store the titles
stans_titles = []

# Loop through the first three pages
for page_num in range(1, 4):
    # Construct the full URL for the current page
    url = f"{base_url}{page_num}/"
    
    # Send a GET request to the webpage
    response = requests.get(url)
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Extract the titles from the 'alt' attribute of 'img' tags
    titles = [img['alt'] for img in soup.find_all('img', alt=True)]
    
    # Add the titles to the list
    stans_titles.extend(titles)

# Display all the extracted titles
stans_titles.remove('lukeinha27')

chichi_url = "https://letterboxd.com/chiarafury/films/rated/4-5/"

# Send a GET request to the webpage
chichi_response = requests.get(chichi_url)
chichi_soup = BeautifulSoup(chichi_response.content, "html.parser")
chichi_titles = [img['alt'] for img in chichi_soup.find_all('img', alt=True)]
chichi_titles.remove('chiarafury')
# chichi_titles


# Collecting the film (target) data
You can use this either to create the target or for the films that are currently on

In [ ]:
first_20_items = clean_target_movies_list[:20]
second_20_items = clean_target_movies_list[21:40]
third_20_items = clean_target_movies_list[41:60]
# Do this for as much target data as you want. The full target list is 180 long but I didn't run it all because of API cost

In [ ]:
# Run this for each of your shortened item list (first_20, second_20 etc) 
# Probably can do code that runs through each. Split it to avoid timeout issues - wouldn't work if you run the whole list at once
# You can also get the imdb rating if you reveal the get ratings code in the middle

def fetch_film_data(film_titles, output_table_name, api_key):
    # RapidAPI details
    api_host = "imdb8.p.rapidapi.com"

    # Base URLs for the APIs
    search_url = "https://imdb8.p.rapidapi.com/v2/search"
    # ratings_url = "https://imdb8.p.rapidapi.com/title/v2/get-ratings"
    keywords_url = "https://imdb8.p.rapidapi.com/title/v2/get-keywords"

    # Headers for the API requests
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": api_host
    }

    # List to store the results
    results = []

    # Loop through the list of film titles
    for title in film_titles:
        # Query string for the search API
        querystring = {"searchTerm": title, "type": "MOVIE", "first": "1", "country": "US", "language": "en-US"}

        # Make the API call to search for the title
        response = requests.get(search_url, headers=headers, params=querystring)
        
        # Adding delay between API calls
        time.sleep(1)
        
        if response.ok:
            try:
                data = response.json()
                edges = data["data"]["mainSearch"]["edges"]
                film_id = None
                for edge in edges:
                    node = edge.get("node", {})
                    entity = node.get("entity", {})
                    if entity.get("__typename") == "Title":
                        film_id = entity.get("id", "")
                        break
            except (KeyError, json.JSONDecodeError) as e:
                print(f"Error processing search response for {title}: {e}")
                film_id = None
        else:
            print(f"Error: {response.status_code} - {response.reason}")
            film_id = None
        
        if film_id:
            # # Make the API call to get ratings
            # querystring2 = {"tconst": film_id, "country": "US", "language": "en-US"}
            # response2 = requests.get(ratings_url, headers=headers, params=querystring2)
            
            # # Adding delay between API calls
            # time.sleep(1)
            
            # if response2.ok:
            #     try:
            #         data2 = response2.json()
            #         aggregate_rating = data2["data"]["title"]["ratingsSummary"]["aggregateRating"]
            #     except (KeyError, json.JSONDecodeError) as e:
            #         print(f"Error processing ratings response for {film_id}: {e}")
            #         aggregate_rating = None
            # else:
            #     print(f"Error: {response2.status_code} - {response2.reason}")
            #     aggregate_rating = None
            
            # # Make the API call to get keywords
            # querystring3 = {"tconst": film_id, "first": "20", "country": "US", "language": "en-US"}
            # response3 = requests.get(keywords_url, headers=headers, params=querystring3)
            
            # # Adding delay between API calls
            # time.sleep(1)
            
            if response3.ok:
                try:
                    data3 = response3.json()
                    keyword_item_categories = data3['data']['title']['keywordItemCategories']
                    keywords = []
                    for category in keyword_item_categories:
                        category_keywords = category.get('keywords', {}).get('edges', [])
                        for edge in category_keywords:
                            keyword_text = edge.get('node', {}).get('keyword', {}).get('text', {}).get('text', '')
                            if keyword_text:
                                keywords.append(keyword_text)
                except (KeyError, json.JSONDecodeError) as e:
                    print(f"Error processing keywords response for {film_id}: {e}")
                    keywords = []
            else:
                print(f"Error: {response3.status_code} - {response3.reason}")
                keywords = []
            
            # Store the result in the results list
            results.append({
                "Title": title,
                "Film ID": film_id,
                # "Aggregate Rating": aggregate_rating,
                "Keywords": keywords
            })
        else:
            print(f"Film ID not found for {title}")

    # Create a DataFrame from the results and save it as a global variable
    globals()[output_table_name] = pd.DataFrame(results)


In [ ]:
# Running the function 
output_table_name = "on_films1"
fetch_film_data(first_20_items, output_table_name, api_key)

# Print the DataFrame
on_films1

In [ ]:
target_rated_film_combo = pd.concat([rated_films1, rated_films2,rated_films3]).reset_index(drop=True)
# Here is using 3 but you can do as many as you want for the target

In [ ]:
# Enriching showing films with keywords

In [ ]:
on_films_joined = pd.concat([on_films1, on_films2,on_films3]).reset_index(drop=True)
# Same for the on film, you can do as many as you want

# Matching the currently showing films to the target 

In [ ]:
nlp = spacy.load('en_core_web_sm')

# Function to preprocess text: tokenize, lemmatize, and remove stop words
def preprocess(text):
    if isinstance(text, list):
        text = ' '.join(text)
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)

# Apply preprocessing to the descriptions
on_films_joined['processed_description'] = on_films_joined['Keywords'].apply(preprocess)
target_rated_film_combo['processed_description'] = target_rated_film_combo['Keywords'].apply(preprocess)

# Combine descriptions with target descriptions
all_keywords = on_films_joined['processed_description'].tolist() + target_rated_film_combo['processed_description'].tolist()

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform all descriptions
tfidf_matrix = vectorizer.fit_transform(all_keywords)

# Separate the tfidf matrix into event descriptions and target descriptions
movie_tfidf_matrix = tfidf_matrix[:len(on_films_joined)]
movie_target_tfidf_matrix = tfidf_matrix[len(on_films_joined):]

# Compute cosine similarity between event descriptions and target descriptions
movie_cosine_sim = cosine_similarity(movie_tfidf_matrix, movie_target_tfidf_matrix)

# Function to get max similarity score for each event description
def get_max_similarity_score(cosine_sim_row):
    return max(cosine_sim_row)

# Iterate through each row and append max similarity score as a new column
on_films_joined['Max_Similarity_Score'] = [get_max_similarity_score(row) for row in movie_cosine_sim]

# Drop the processed_description column as it's not needed for the final output
on_films_joined = on_films_joined.drop(columns=['processed_description'])

# Display the DataFrame with the new column
# on_films_joined

threshold2 = 0.3
recommended_films = on_films_joined[on_films_joined['Max_Similarity_Score'] > threshold2]

recommended_films = recommended_films[recommended_films['Aggregate Rating'] > 6.4]
# recommended_films

You need to join your tables, 
- recommended_films is your main table and you add the image url from top_rated_films 
- alternatively change your code so that you only use one api to retrieve the id and then use both other apis the second time around and retrieve keywords, rating and image url together

Then you need to generate a link for the button 

In [ ]:
import urllib.parse

base_url = "https://www.yorck.de/en/films/"

# Function to generate link from title
def generate_link(title):
    # Clean title, convert to lowercase, and replace spaces with hyphens
    cleaned_title = title.strip().lower().replace(" ", "-")
    # Encode the title to handle special characters
    encoded_title = urllib.parse.quote(cleaned_title)
    # Concatenate the base URL and encoded title
    link = base_url + encoded_title + "?"
    return link


# Apply the function to generate links
recommended_films['Link'] = recommended_films['Title'].apply(generate_link)

# Get Plot Data

In [ ]:
def fetch_film_data(film_ids, api_key):
    api_host = "imdb8.p.rapidapi.com"
    plots_url = "https://imdb8.p.rapidapi.com/title/v2/get-plots"
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": api_host
    }

    titles_list = []
    plots_list = []
    summaries_list = []

    for film_id in film_ids:
        success = False
        retries = 3
        delay = 1  # Start with 1 second delay

        while not success and retries > 0:
            try:
                querystring = {"tconst": film_id}
                response = requests.get(plots_url, headers=headers, params=querystring)
                
                if response.status_code == 403:
                    print(f"Access forbidden for film ID {film_id}: Check your API key and permissions.")
                    titles_list.append('N/A')
                    plots_list.append('N/A')
                    summaries_list.append([])
                    success = True
                elif response.status_code == 429:
                    print(f"Rate limit hit for film ID {film_id}: Retrying after delay.")
                    retries -= 1
                    time.sleep(delay)
                    delay *= 2  # Exponential backoff
                elif response.ok:
                    data = response.json()

                    # Debugging the structure of the JSON response
                    print(f"Response for {film_id}: {data}")

                    title = data.get('data', {}).get('title', {}).get('titleText', {}).get('text', 'N/A')
                    plot = data.get('data', {}).get('title', {}).get('plot', {}).get('plotText', {}).get('plainText', 'N/A')
                    plots_edges = data.get('data', {}).get('title', {}).get('plots', {}).get('edges', [])
                    summaries = [edge.get('node', {}).get('plotText', {}).get('plainText', 'N/A') for edge in plots_edges]

                    titles_list.append(title)
                    plots_list.append(plot)
                    summaries_list.append(summaries)
                    success = True
                else:
                    print(f"Error fetching data for film ID {film_id}: {response.status_code}")
                    titles_list.append('N/A')
                    plots_list.append('N/A')
                    summaries_list.append([])
                    success = True
            except Exception as e:
                print(f"Error fetching data for film ID {film_id}: {e}")
                titles_list.append('N/A')
                plots_list.append('N/A')
                summaries_list.append([])
                success = True

    return pd.DataFrame({
        'Film ID': film_ids,
        'Title': titles_list,
        'Plot': plots_list,
        'Summaries': summaries_list
    })

# Example usage

film_ids = recommended_films['Film ID'].tolist()  # Assuming 'recommended_films' is your input DataFrame

# Fetch data and create new DataFrame
plot_data = fetch_film_data(film_ids, api_key)
plot_data = plot_data.drop(columns=['Summaries'])
plot_data = plot_data.rename(columns={'Title': 'OGTitle'})
recommended_films_final = recommended_films
recommended_films_mega_final = recommended_films_final.merge(plot_data, on='Film ID', how='left')

In [ ]:
recommended_films_final.to_json('film_recs.json', orient='records')